In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
#Instalar Kaggle para posteriormente descargar el dataset de imagenes
!pip install kaggle


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#Subir el json con la clave api de kagle

from google.colab import files

uploaded = files.upload()


Saving kaggle.json to kaggle.json


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
#Descargar datasets

!kaggle datasets download -d erhmrai/ecg-image-data


100% 857M/858M [00:44<00:00, 23.7MB/s]
100% 858M/858M [00:44<00:00, 20.3MB/s]


In [6]:
#Los descomprimimos

import zipfile

with zipfile.ZipFile('/content/ecg-image-data.zip', 'r') as zip_ref:
    zip_ref.extractall('.')


In [7]:
!pip install split-folders


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [8]:
!mkdir -p /content/ECG_Image_data/train_val

In [9]:
#Dividir la carpeta de train en train y val

import splitfolders

input_folder = "/content/ECG_Image_data/train"
output_folder = "/content/ECG_Image_data/train_val"
split_ratio = (0.8, 0.2)  # 80% para entrenamiento y 20% para validación

splitfolders.ratio(input_folder, output=output_folder, seed=42, ratio=split_ratio, group_prefix=None)


Copying files: 99199 files [00:12, 7941.99 files/s]


###Como la entrada de datos del modelo de imágenes ECG llegarán tras procesarse en blanco y negro, convertimos todas las imagenes de kagle a escala de grises para entrenar mejor el modelo. Tenemos el inconveniente que el conjunto de datos de imagenet no acepta 1 solo canal así que le daremos a cada imagen un seudo color, es decir le añadiremos dos canales más para que por formato el modelo las entrene correctamente.

In [10]:
from PIL import Image
import os

#TRAIN
input_folder = '/content/ECG_Image_data/train_val/train'
output_folder = '/content/train_final'


# Crear la carpeta de salida si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterar sobre las subcarpetas
for subfolder in os.listdir(input_folder):
    subfolder_path = os.path.join(input_folder, subfolder)
    
    # Comprobar si es una carpeta
    if os.path.isdir(subfolder_path):
        output_subfolder = os.path.join(output_folder, subfolder)
        
        # Crear la subcarpeta de salida si no existe
        if not os.path.exists(output_subfolder):
            os.makedirs(output_subfolder)
        
        # Iterar sobre las imágenes en la subcarpeta
        for filename in os.listdir(subfolder_path):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                img_color = Image.open(os.path.join(subfolder_path, filename))
                img_gray = img_color.convert('L')
                img_gray.save(os.path.join(output_subfolder, filename))



In [11]:
#VALIDATION

input_folder = '/content/ECG_Image_data/train_val/val'
output_folder = '/content/validation_final'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterar sobre las subcarpetas
for subfolder in os.listdir(input_folder):
    subfolder_path = os.path.join(input_folder, subfolder)
    
    # Comprobar si es una carpeta
    if os.path.isdir(subfolder_path):
        output_subfolder = os.path.join(output_folder, subfolder)
        
        # Crear la subcarpeta de salida si no existe
        if not os.path.exists(output_subfolder):
            os.makedirs(output_subfolder)
        
        # Iterar sobre las imágenes en la subcarpeta
        for filename in os.listdir(subfolder_path):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                img_color = Image.open(os.path.join(subfolder_path, filename))
                img_gray = img_color.convert('L')
                img_gray.save(os.path.join(output_subfolder, filename))


In [12]:
#TEST
input_folder = '/content/ECG_Image_data/test'
output_folder = '/content/test_final'

# Crear la carpeta de salida si no existe
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterar sobre las subcarpetas
for subfolder in os.listdir(input_folder):
    subfolder_path = os.path.join(input_folder, subfolder)
    
    # Comprobar si es una carpeta
    if os.path.isdir(subfolder_path):
        output_subfolder = os.path.join(output_folder, subfolder)
        
        # Crear la subcarpeta de salida si no existe
        if not os.path.exists(output_subfolder):
            os.makedirs(output_subfolder)
        
        # Iterar sobre las imágenes en la subcarpeta
        for filename in os.listdir(subfolder_path):
            if filename.endswith(".jpg") or filename.endswith(".png"):
                img_color = Image.open(os.path.join(subfolder_path, filename))
                img_gray = img_color.convert('L')
                img_gray.save(os.path.join(output_subfolder, filename))

In [13]:
#Pruebo a eliminar la clase M ya que no encontramos datos sobre a que puede referirse a diferencia de las otras clases
!rm -r /content/test_final/M
!rm -r /content/train_final/M
!rm -r /content/validation_final/M



In [26]:
#Movemos las imagenes de una carpeta a otra
!mv /content/drive/MyDrive/ECG.Training/latidos_F/* /content/train_final/F
!mv /content/drive/MyDrive/ECG.Training/latidos_N/* /content/train_final/N
!mv /content/drive/MyDrive/ECG.Training/latidos_Q/* /content/train_final/Q
!mv /content/drive/MyDrive/ECG.Training/latidos_S/* /content/train_final/S
!mv /content/drive/MyDrive/ECG.Training/latidos_V/* /content/train_final/V




In [25]:
!rm -r /content/train_final/V/latidos_V


In [29]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, Flatten

# Función para convertir imágenes en escala de grises a pseudo-color

def custom_preprocessing(img_gray):
    img_gray = 0.2989 * img_gray[:, :, 0] + 0.5870 * img_gray[:, :, 1] + 0.1140 * img_gray[:, :, 2]
    img_pseudocolor = np.stack((img_gray,)*3, axis=-1)
    img_preprocessed = preprocess_input(img_pseudocolor*255)
    return img_preprocessed



# Rutas de las carpetas de train, validation y test
train_folder = '/content/train_final'
val_folder = '/content/validation_final'
test_folder = '/content/test_final'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest',
    preprocessing_function=custom_preprocessing
)

val_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=custom_preprocessing
)

test_datagen = ImageDataGenerator(
    rescale=1./255,
    preprocessing_function=custom_preprocessing
)


# Generadores para los conjuntos de entrenamiento, validación y prueba
train_generator = train_datagen.flow_from_directory(
    train_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)

val_generator = val_datagen.flow_from_directory(
    val_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)

test_generator = test_datagen.flow_from_directory(
    test_folder,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    color_mode='rgb'
)
base_model = vgg16.VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(5, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

EPOCHS = 5
model_ecg = model.fit(train_generator, validation_data= val_generator, epochs=EPOCHS, steps_per_epoch=len(train_generator), validation_steps=len(val_generator))



Found 72841 images belonging to 5 classes.
Found 18161 images belonging to 5 classes.
Found 22698 images belonging to 5 classes.
58889256/58889256 [==============================] - 3s 0us/step
Epoch 1/5
2277/2277 [==============================] - 854s 369ms/step - loss: 0.6754 - accuracy: 0.9167 - val_loss: 0.1059 - val_accuracy: 0.9677
Epoch 2/5
2277/2277 [==============================] - 837s 367ms/step - loss: 0.2091 - accuracy: 0.9386 - val_loss: 0.1118 - val_accuracy: 0.9665
Epoch 3/5
2277/2277 [==============================] - 837s 368ms/step - loss: 0.1900 - accuracy: 0.9433 - val_loss: 0.1084 - val_accuracy: 0.9681
Epoch 4/5
2277/2277 [==============================] - 837s 368ms/step - loss: 0.1769 - accuracy: 0.9447 - val_loss: 0.1182 - val_accuracy: 0.9683
Epoch 5/5
2277/2277 [==============================] - 832s 365ms/step - loss: 0.1729 - accuracy: 0.9467 - val_loss: 0.0933 - val_accuracy: 0.9738


In [30]:
model.save('/content/drive/MyDrive/saved_models/model_ecg_escala_grises_mejorado.h5')


In [31]:
train_generator.class_indices

{'F': 0, 'N': 1, 'Q': 2, 'S': 3, 'V': 4}

###Observamos que las predicciones no son correctas. Aunque tras una profunda revisión vemos que el modelo parece estar bien entrenado, el problema es como le estamos haciendo llegar los datos al modelo. Probamos a reentrenar el modelo añadiendole imágenes distintas de los distintos tipos de latidos pero sigue realizando de manera incorrecta las predicciones.

##PRUEBA MODELO IMAGENES A COLOR


Tras probar este modelo, nos encontramos que las predicciones siguen siendo erróneas. Viendo el poco tiempo del que disponemos omitimos seguir investigando como optimizar el modelo de imágenes y continuamos el estudio con el modelo numérico.

In [28]:
#Creamos otro modelo mas simple que procesa imágenes a color y verificamos resultados.

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import vgg16

In [ ]:
train_dir = '/content/ECG_Image_data/train_val/train'
val_dir = '/content/ECG_Image_data/train_val/val'
test_dir = '/content/ECG_Image_data/test'

train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=10,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   shear_range=0.1,
                                   zoom_range=0.1,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

val_datagen = ImageDataGenerator(rescale=1./255)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir,
                                                    target_size=(224, 224),
                                                    color_mode='rgb',
                                                    batch_size=64,
                                                    class_mode='categorical')

val_generator = val_datagen.flow_from_directory(val_dir,
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=64,
                                                class_mode='categorical')

test_generator = test_datagen.flow_from_directory(test_dir,
                                                target_size=(224, 224),
                                                color_mode='rgb',
                                                batch_size=64,
                                                class_mode='categorical')





Found 79357 images belonging to 6 classes.
Found 19842 images belonging to 6 classes.
Found 24799 images belonging to 6 classes.


In [29]:
base_model = vgg16.VGG16(input_shape=(224, 224, 3), include_top=False, weights='imagenet')

for layer in base_model.layers:
    layer.trainable = False

x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(6, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
EPOCHS = 5
model_ecg = model.fit(train_generator, validation_data= val_generator, epochs=EPOCHS, steps_per_epoch=len(train_generator), validation_steps=len(val_generator))
model.save('/content/drive/MyDrive/saved_models/model_ecg_escala_grises.h5')


In [ ]:
model.save('/content/drive/MyDrive/saved_models/model_ecg_escala_grises.h5')